# NAS - Optuna

- **Authored by:** Matheus Ferreira Silva 
- **GitHub:**: https://github.com/MatheusFS-dev

## 1. Setup and Configuration

### 1.1. Environment Variables

In [1]:
import os

# Async CUDA allocator
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

# If cuDNN autotune fails, fall back to a safe (but slower) algorithm.
os.environ["XLA_FLAGS"] = "--xla_gpu_strict_conv_algorithm_picker=false"

# Allow TensorFlow to allocate GPU memory as needed
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 

### 1.2. Imports

In [2]:
from _imports import * # Centralized file containing all imports

2025-05-18 10:30:56.561074: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-18 10:30:56.574101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747575056.586999   20089 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747575056.590198   20089 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-18 10:30:56.604774: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### 1.3. GPU Management

In [3]:
# Specify GPU to use (e.g., GPU 0)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
troo.get_gpu_info()

TensorFlow Version: 2.18.0
CUDA support detected
  CUDA Version: 12.5.1
  cuDNN Version: 9

GPUs Detected (1): ['/physical_device:GPU:0']
Default GPU device: /device:GPU:0


2025-05-18 10:30:58.668530: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
I0000 00:00:1747575058.668583   20089 gpu_process_state.cc:201] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1747575058.668842   20089 gpu_device.cc:2022] Created device /device:GPU:0 with 2229 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


## 2. Run Parameters 

In [4]:
TOTAL_NUM_PORTS = 100

observed_ports_list = [3, 4, 5, 6, 7, 10, 15]
batch_sizes = [64, 64, 128, 64, 64, 256, 128]

model_paths = [
    "./results/models/cnn/optuna_study_3_ports/models/trial_134.keras",
    "./results/models/cnn/optuna_study_4_ports/models/trial_139.keras",
    "./results/models/cnn/optuna_study_5_ports/models/trial_178.keras",
    "./results/models/cnn/optuna_study_6_ports/models/trial_79.keras",
    "./results/models/cnn/optuna_study_7_ports/models/trial_183.keras",
    "./results/models/cnn/optuna_study_10_ports/models/trial_125.keras",
    "./results/models/cnn/optuna_study_15_ports/models/trial_183.keras",
]

scalers = [
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
    StandardScaler(),
]


THRESHOLD = 0.95
SNR_LINEAR = 1.25

mixed_precision.set_global_policy("mixed_float16")

In [5]:
RUN_DIR = troo.create_run_directory(prefix="cnn_op_")
print(f"Run directory: {RUN_DIR}")

Run directory: runs/cnn_op_1


## 3. Data Loading and Preprocessing

In [6]:
# --------------------- Load the dataset in matlab format -------------------- #
rng = np.random.default_rng(42)

kappa0_mu1_m0 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu1.0_m0.0.mat")["SNR_events"]
kappa0_mu1_m2 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu1.0_m2.0.mat")["SNR_events"]
kappa0_mu1_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu1.0_m50.0.mat")["SNR_events"]
kappa0_mu2_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu2.0_m50.0.mat")["SNR_events"]
kappa0_mu5_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa1.0e-16_mu5.0_m50.0.mat")["SNR_events"]
kappa5_mu1_m0 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu1.0_m0.0.mat")["SNR_events"]
kappa5_mu1_m2 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu1.0_m2.0.mat")["SNR_events"]
kappa5_mu1_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu1.0_m50.0.mat")["SNR_events"]
kappa5_mu2_m0 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu2.0_m0.0.mat")["SNR_events"]
kappa5_mu2_m2 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu2.0_m2.0.mat")["SNR_events"]
kappa5_mu2_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu2.0_m50.0.mat")["SNR_events"]
kappa5_mu5_m0 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu5.0_m0.0.mat")["SNR_events"]
kappa5_mu5_m2 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu5.0_m2.0.mat")["SNR_events"]
kappa5_mu5_m50 = scipy.io.loadmat("./data/w1_u1_n100/SNR_events_W1.0_U1_N100_kappa5.0e+00_mu5.0_m50.0.mat")["SNR_events"]

# ————————————— Split the data into 10% training and 90% testing ————————————— #

# kappa0_mu1_m0
perm = rng.permutation(kappa0_mu1_m0.shape[0])
n_test = int(0.9*kappa0_mu1_m0.shape[0])
kappa0_mu1_m0_test = kappa0_mu1_m0[perm[:n_test]]
kappa0_mu1_m0 = kappa0_mu1_m0[perm[n_test:]]

# kappa0_mu1_m2
perm = rng.permutation(kappa0_mu1_m2.shape[0])
n_test = int(0.9*kappa0_mu1_m2.shape[0])
kappa0_mu1_m2_test = kappa0_mu1_m2[perm[:n_test]]
kappa0_mu1_m2 = kappa0_mu1_m2[perm[n_test:]]

# kappa0_mu1_m50
perm = rng.permutation(kappa0_mu1_m50.shape[0])
n_test = int(0.9*kappa0_mu1_m50.shape[0])
kappa0_mu1_m50_test = kappa0_mu1_m50[perm[:n_test]]
kappa0_mu1_m50 = kappa0_mu1_m50[perm[n_test:]]

# kappa0_mu2_m50
perm = rng.permutation(kappa0_mu2_m50.shape[0])
n_test = int(0.9*kappa0_mu2_m50.shape[0])
kappa0_mu2_m50_test = kappa0_mu2_m50[perm[:n_test]]
kappa0_mu2_m50 = kappa0_mu2_m50[perm[n_test:]]

# kappa0_mu5_m50
perm = rng.permutation(kappa0_mu5_m50.shape[0])
n_test = int(0.9*kappa0_mu5_m50.shape[0])
kappa0_mu5_m50_test = kappa0_mu5_m50[perm[:n_test]]
kappa0_mu5_m50 = kappa0_mu5_m50[perm[n_test:]]

# kappa5_mu1_m0
perm = rng.permutation(kappa5_mu1_m0.shape[0])
n_test = int(0.9*kappa5_mu1_m0.shape[0])
kappa5_mu1_m0_test = kappa5_mu1_m0[perm[:n_test]]
kappa5_mu1_m0 = kappa5_mu1_m0[perm[n_test:]]

# kappa5_mu1_m2
perm = rng.permutation(kappa5_mu1_m2.shape[0])
n_test = int(0.9*kappa5_mu1_m2.shape[0])
kappa5_mu1_m2_test = kappa5_mu1_m2[perm[:n_test]]
kappa5_mu1_m2 = kappa5_mu1_m2[perm[n_test:]]

# kappa5_mu1_m50
perm = rng.permutation(kappa5_mu1_m50.shape[0])
n_test = int(0.9*kappa5_mu1_m50.shape[0])
kappa5_mu1_m50_test = kappa5_mu1_m50[perm[:n_test]]
kappa5_mu1_m50 = kappa5_mu1_m50[perm[n_test:]]

# kappa5_mu2_m0
perm = rng.permutation(kappa5_mu2_m0.shape[0])
n_test = int(0.9*kappa5_mu2_m0.shape[0])
kappa5_mu2_m0_test = kappa5_mu2_m0[perm[:n_test]]
kappa5_mu2_m0 = kappa5_mu2_m0[perm[n_test:]]

# kappa5_mu2_m2
perm = rng.permutation(kappa5_mu2_m2.shape[0])
n_test = int(0.9*kappa5_mu2_m2.shape[0])
kappa5_mu2_m2_test = kappa5_mu2_m2[perm[:n_test]]
kappa5_mu2_m2 = kappa5_mu2_m2[perm[n_test:]]

# kappa5_mu2_m50
perm = rng.permutation(kappa5_mu2_m50.shape[0])
n_test = int(0.9*kappa5_mu2_m50.shape[0])
kappa5_mu2_m50_test = kappa5_mu2_m50[perm[:n_test]]
kappa5_mu2_m50 = kappa5_mu2_m50[perm[n_test:]]

# kappa5_mu5_m0
perm = rng.permutation(kappa5_mu5_m0.shape[0])
n_test = int(0.9*kappa5_mu5_m0.shape[0])
kappa5_mu5_m0_test = kappa5_mu5_m0[perm[:n_test]]
kappa5_mu5_m0 = kappa5_mu5_m0[perm[n_test:]]

# kappa5_mu5_m2
perm = rng.permutation(kappa5_mu5_m2.shape[0])
n_test = int(0.9*kappa5_mu5_m2.shape[0])
kappa5_mu5_m2_test = kappa5_mu5_m2[perm[:n_test]]
kappa5_mu5_m2 = kappa5_mu5_m2[perm[n_test:]]

# kappa5_mu5_m50
perm = rng.permutation(kappa5_mu5_m50.shape[0])
n_test = int(0.9*kappa5_mu5_m50.shape[0])
kappa5_mu5_m50_test = kappa5_mu5_m50[perm[:n_test]]
kappa5_mu5_m50 = kappa5_mu5_m50[perm[n_test:]]

# ————————————— Concatenate all training subsamples along axis=0 ————————————— #
dataset = np.concatenate(
    [
        kappa0_mu1_m0,
        kappa0_mu1_m2,
        kappa0_mu1_m50,
        kappa0_mu2_m50,
        kappa0_mu5_m50,
        kappa5_mu1_m0,
        kappa5_mu1_m2,
        kappa5_mu1_m50,
        kappa5_mu2_m0,
        kappa5_mu2_m2,
        kappa5_mu2_m50,
        kappa5_mu5_m0,
        kappa5_mu5_m2,
        kappa5_mu5_m50,
    ],
    axis=0,
)

print(f"Original dataset shape: {dataset.shape}")

# Subsample data
# dataset = dataset[: int(0.01 * dataset.shape[0]), :]

print(f"Shape of the data after configuration: {dataset.shape}\n")

Original dataset shape: (1400000, 100)
Shape of the data after configuration: (1400000, 100)



## 4. Getters

### 4.6. Implementation getters

In [7]:
def get_observed_ports(sinr_data, num_observed_ports, total_ports):
    """
    Extracts SINR values for the specified number of observed ports.

    The function selects a subset of SINR data by identifying equally spaced ports based on the
    number of observed ports specified. It returns the SINR values for these observed ports and
    their corresponding indices.

    Args:
        sinr_data (numpy.ndarray): A 2D array where each row represents an observation and each column
                                   represents a port with its corresponding SINR values.
        num_observed_ports (int): The number of observed ports to select from the SINR data.
        total_ports (int): The total number of ports in the SINR data.

    Returns:
        observed_sinr (numpy.ndarray): A 2D array containing the SINR values for the observed ports.
        observed_indices (numpy.ndarray): A 1D array of the indices corresponding to the observed ports.
    """
    observed_indices = np.linspace(0, total_ports - 1, num_observed_ports, dtype=int)
    observed_sinr = sinr_data[:, observed_indices]

    return observed_sinr, observed_indices


def getOP(
    observed_indices: np.ndarray,
    predicted_values: np.ndarray,
    true_values: np.ndarray,
    threshold: float,
    snr_linear: float,
    total_ports: int,
) -> float:
    """Estimate the outage probability for regression models.

    This function compares the predicted and observed signal values at different
    channels (ports) and determines whether the chosen signal is above a given threshold.
    The outage probability is then computed as the proportion of times the signal falls
    below this threshold.

    Args:
        observed_indices (np.ndarray): Indices of the observed ports (channels).
        predicted_values (np.ndarray): Matrix of predicted values for each sample.
        true_values (np.ndarray): Ground-truth values for each port.
        threshold (float): Threshold value for determining outage.
        snr_linear (float): Signal-to-noise ratio in linear scale.

    Returns:
        float: Estimated outage probability.
    """

    # Initialize an array with negative infinity to store the observed values
    observed_values_matrix = np.full((true_values.shape[0], total_ports), -np.inf, dtype=np.float64)

    # Assign the true values of the observed ports (channels) to the matrix
    observed_values_matrix[:, observed_indices] = true_values[:, observed_indices]

    # Find the index of the highest predicted value for each sample
    best_predicted_indices = np.argmax(predicted_values, axis=1)

    # Initialize an array with negative infinity to store the predicted values
    predicted_values_matrix = np.full((true_values.shape[0], total_ports), -np.inf, dtype=np.float64)

    # Assign the true value corresponding to the predicted best port
    predicted_values_matrix[np.arange(len(best_predicted_indices)), best_predicted_indices] = true_values[
        np.arange(len(best_predicted_indices)), best_predicted_indices
    ]

    # Take the element-wise maximum between the observed and predicted value matrices
    best_value_matrix = np.maximum(observed_values_matrix, predicted_values_matrix)

    # print("Shape of Best Value Matrix:", best_value_matrix.shape)

    # Find the index of the best predicted or observed port (channel) for each sample
    best_predicted_or_observed_ports = np.argmax(best_value_matrix, axis=1)

    # print("Shape of Best Predicted/Observed Ports:", best_predicted_or_observed_ports.shape)
    # print("Number of Selected Ports:", len(best_predicted_or_observed_ports))

    # Retrieve the actual values corresponding to the best selected ports
    selected_values = best_value_matrix[np.arange(len(true_values)), best_predicted_or_observed_ports]

    # print("Shape of Selected Values:", selected_values.shape)

    # Determine which selected values are above the given threshold
    above_threshold = selected_values > (threshold / snr_linear)

    # print("Shape of Above Threshold Array:", above_threshold.shape)

    # Compute the outage probability: probability that the selected value is below the threshold
    outage_probability = 1.0 - (np.sum(above_threshold) / len(true_values))

    return outage_probability


def getObservedOP(
    observed_indices: np.ndarray, true_values: np.ndarray, threshold: float, snr_linear: float
) -> float:
    """
    Outage probability when you only observe a subset of ports.

    For each sample, picks the best SINR among observed ports,
    then computes OP = 1 - P(best_obs > threshold/snr_linear).
    """
    # extract only observed-port SINRs
    observed_sinr = true_values[:, observed_indices]
    # best per sample
    best_obs = np.max(observed_sinr, axis=1)
    # fraction above threshold
    p_above = np.mean(best_obs > (threshold / snr_linear))
    return 1.0 - p_above


def getIdealOP(true_values: np.ndarray, threshold: float, snr_linear: float) -> float:
    """
    Genie‐aided outage probability knowing all ports.

    For each sample, picks the best SINR across all ports,
    then computes OP = 1 - P(best_all > threshold/snr_linear).
    """
    best_all = np.max(true_values, axis=1)
    p_above = np.mean(best_all > (threshold / snr_linear))
    return 1.0 - p_above

## MAIN 

In [8]:
datasets = [
    kappa0_mu1_m0_test,
    kappa0_mu1_m2_test,
    kappa0_mu1_m50_test,
    kappa0_mu2_m50_test,
    kappa0_mu5_m50_test,
    kappa5_mu1_m0_test,
    kappa5_mu1_m2_test,
    kappa5_mu1_m50_test,
    kappa5_mu2_m0_test,
    kappa5_mu2_m2_test,
    kappa5_mu2_m50_test,
    kappa5_mu5_m0_test,
    kappa5_mu5_m2_test,
    kappa5_mu5_m50_test,
]

dataset_names: list[str] = [
    "kappa0_mu1_m0_test",
    "kappa0_mu1_m2_test",
    "kappa0_mu1_m50_test",
    "kappa0_mu2_m50_test",
    "kappa0_mu5_m50_test",
    "kappa5_mu1_m0_test",
    "kappa5_mu1_m2_test",
    "kappa5_mu1_m50_test",
    "kappa5_mu2_m0_test",
    "kappa5_mu2_m2_test",
    "kappa5_mu2_m50_test",
    "kappa5_mu5_m0_test",
    "kappa5_mu5_m2_test",
    "kappa5_mu5_m50_test",
]

In [9]:
# Precompute ideal OP once per dataset
ideal_ops_global = [
    getIdealOP(data, THRESHOLD, SNR_LINEAR)
    for data in datasets
]

for n_ports, model_path, batch_size, scaler in zip(observed_ports_list, model_paths, batch_sizes, scalers):
    # create and/or clear subfolder
    sub_dir = os.path.join(RUN_DIR, f"op_{n_ports}_observed_ports")
    os.makedirs(sub_dir, exist_ok=True)

    # load the single model for this n_ports
    model = tf.keras.models.load_model(model_path)

    # fit scaler on training split observed at n_ports
    observed_ports, _ = get_observed_ports(dataset, n_ports, TOTAL_NUM_PORTS)
    X_train, X_val, y_train, y_val = train_test_split(
        observed_ports, dataset, test_size=0.2, random_state=0, shuffle=True
    )
    scaler.fit(X_train)

    test_losses = []
    ops = []
    obs_ops = []

    # evaluate on each test subset
    for data, name in zip(datasets, dataset_names):
        X_test, idxs = get_observed_ports(data, n_ports, TOTAL_NUM_PORTS)
        X_test = scaler.transform(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)
        loss = model.evaluate(X_test, data, batch_size=batch_size, verbose=1)
        test_losses.append(loss)

        y_pred = model.predict(X_test, verbose=1)
        op = getOP(idxs, y_pred, data, THRESHOLD, SNR_LINEAR, TOTAL_NUM_PORTS)
        ops.append(op)
        
        # compute observed-only OP
        obs_op = getObservedOP(idxs, data, THRESHOLD, SNR_LINEAR)
        obs_ops.append(obs_op)

    # print a concise summary
    print(f"\n=== {n_ports} observed ports ===")
    for name, loss, op, obs_op, ideal_op in zip(
        dataset_names, test_losses, ops, obs_ops, ideal_ops_global
    ):
        print(f"{name}: Loss={loss:.6f}, OP={op:.6f}, ObsOP={obs_op:.6f}")

    # save results to file
    results = {f"{n}_loss": l for n, l in zip(dataset_names, test_losses)}
    results.update({f"{n}_op": o for n, o in zip(dataset_names, ops)})
    results.update({f"{n}_obsOP": o for n, o in zip(dataset_names, obs_ops)})

    out_file = os.path.join(sub_dir, f"results_{n_ports}_ports.txt")
    troo.save_trial_params_to_file(filepath=out_file, params={}, **results)
    
# 3. Global ideal-OP section (once per dataset)
print("\n=== Ideal Outage Probability (genie-aided) per dataset ===")
for name, ideal_op in zip(dataset_names, ideal_ops_global):
    print(f"{name}: IdealOP={ideal_op:.6f}")

# (Optional) save to a dedicated file
ideal_results = {f"{n}_idealOP": o for n, o in zip(dataset_names, ideal_ops_global)}
ideal_file = os.path.join(RUN_DIR, "ideal_ops_global.txt")
troo.save_trial_params_to_file(filepath=ideal_file, params={}, **ideal_results)

I0000 00:00:1747575156.537679   20089 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2229 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 18 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 14 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
I0000 00:00:1747575159.188652   20302 service.cc:148] XLA service 0x760298004dc0 initialized for platform CUDA (this does not guarantee 

  123/14063 ━━━━━━━━━━━━━━━━━━━━ 12s 932us/step - loss: 0.4738

I0000 00:00:1747575159.718038   20302 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14063/14063 ━━━━━━━━━━━━━━━━━━━━ 11s 733us/step - loss: 0.4467
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 21s 738us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 9s 653us/step - loss: 0.4433
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 15s 519us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 8s 595us/step - loss: 0.4456
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 18s 637us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 9s 609us/step - loss: 0.3602
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 621us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 10s 688us/step - loss: 0.3001
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 18s 634us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 9s 604us/step - loss: 25.8076
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 18s 626us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 9s 608us/step - loss: 0.3886
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 18s 643us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 9s 612us/step - loss: 0.3252
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 18s 644us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 9s 638us/step - loss: 23.8984
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 19s 683us/step
14063/

/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 18 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 14 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


...

14063/14063 ━━━━━━━━━━━━━━━━━━━━ 11s 766us/step - loss: 1.7511e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 22s 769us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 11s 799us/step - loss: 1.7365e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 19s 663us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 10s 717us/step - loss: 1.7432e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 19s 666us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 10s 697us/step - loss: 1.2629e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 726us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 11s 774us/step - loss: 1.1327e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 724us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 12s 826us/step - loss: 0.0123
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 705us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 11s 799us/step - loss: 1.3532e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 20s 724us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 10s 714us/step - loss: 1.1643e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 19s 665us/step
14063/14063 ━━━━━━━━━━━━━━━━━━━━ 10s 715us/step - loss: 0.0075
28125/28125 ━━━━━━━━━

/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 18 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 14 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


3516/3516 ━━━━━━━━━━━━━━━━━━━━ 4s 988us/step - loss: 1.7924e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 15s 530us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 932us/step - loss: 1.7822e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 15s 523us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 928us/step - loss: 1.7774e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 15s 517us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 943us/step - loss: 1.1905e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 15s 518us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 9.8655e-05
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 22s 782us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 976us/step - loss: 0.0334
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 16s 580us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 4s 990us/step - loss: 1.3137e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 618us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 964us/step - loss: 1.0577e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 17s 599us/step
3516/3516 ━━━━━━━━━━━━━━━━━━━━ 3s 954us/step - loss: 0.0210
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 16s 580us/step
35

/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 18 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/home/matheus/anaconda3/envs/tf-optuna/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 14 variables whereas the saved optimizer has 0 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


7032/7032 ━━━━━━━━━━━━━━━━━━━━ 7s 889us/step - loss: 4.8018e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 14s 494us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 6s 841us/step - loss: 4.7733e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 13s 470us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 6s 875us/step - loss: 4.7836e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 14s 497us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 6s 878us/step - loss: 3.4499e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 15s 536us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 6s 844us/step - loss: 2.7937e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 14s 502us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 6s 849us/step - loss: 0.0389
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 14s 503us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 6s 851us/step - loss: 3.8022e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 14s 498us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 6s 850us/step - loss: 3.0530e-04
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 14s 511us/step
7032/7032 ━━━━━━━━━━━━━━━━━━━━ 6s 866us/step - loss: 0.0242
28125/28125 ━━━━━━━━━━━━━━━━━━━━ 15s 530us/step
